# Отчет по заданию

## Задание №1: 
<br/>
Есть две таблицы:

***data_mortage***

client | contract
:------|:--------
1 | MCC1203 
2 | MZS1505
3 | MXX566

и

***data_spravochnik***

code | type
:----|:----
MCC | потреб 
MZS | ипотека
MAA | автокредит

Нужно определить у какого клиента какой кредит. 

## Решение Задания №1:
<br/>
У меня под боком есть сервер с ClickHouse. Буду делать на нем.<br/>
Создаю базу, таблицы, и наполню их данными:

``` mysql
CREATE DATABASE IF NOT EXISTS pbank;

CREATE TABLE pbank.data_mortage
(
client Int64,
contract String
) ENGINE = MergeTree ORDER BY (client) SETTINGS index_granularity = 8192;

insert into pbank.data_mortage
values
('1', 'MCC1203'),
('2', 'MZS1505'),
('3', 'MXX566');


CREATE TABLE pbank.data_spravochnik
(
code String,
type String
) ENGINE = MergeTree ORDER BY (code) SETTINGS index_granularity = 8192;

insert into pbank.data_spravochnik
values
('MCC', 'потреб'),
('MZS', 'ипотека'),
('MAA', 'автокредит');

```
<br/>
<br/>
К сожалению, ClickHouse в join позволяет использовать только условие '=' между значениями из разных таблиц. Нет возможности использовать 'like'. Функции extract и like не позволяют указать в качестве регулярного выражения имя столбца. Но всегда есть обходные пути. 
<br/>
<br/>
Код SQL запроса:
<br/>

``` mysql
select
    client,
    contract,
    CASE
    WHEN empty(r.contract) THEN 'не найдено в справочнике'
    ELSE type
    END as type
from
    pbank.data_mortage as l
left join
    (
    select
        contract,
        type
    from
        pbank.data_mortage
    cross join
        pbank.data_spravochnik
    where position(contract, code) > 0
    ) as r
on
    l.contract = r.contract
```
<br/>
<br/>
Результирующая таблица на скрине внизу:
<br/>

![01.jpg](pics/01.jpg)

## Задание №2: 
<br/>
Дана таблица:

***data_zad2***

id | Date | Marker
:--|:---- |:-------
1 | 2019-04-21 | X 
1 | 2019-07-01 |  
1 | 2019-05-19 | X 
1 | 2019-05-01 | X 
2 | 2019-05-12 | X 
2 | 2019-05-24 |  
2 | 2019-05-18 | X 
2 | 2019-05-30 | X 
3 | 2019-02-21 | X 
3 | 2019-05-15 | X 
3 | 2019-05-05 |  
3 | 2019-02-21 | 
3 | 2019-05-11 | X


Нужно определить первую и последнюю дату по каждому клиенту с проставленным критерием.

## Решение Задания №2:
<br/>
Создаю таблицу, и наполню ее данными:

``` mysql
CREATE TABLE pbank.data_zad2
(
id Int64,
Date Date,
Marker String
) ENGINE = MergeTree ORDER BY (id) SETTINGS index_granularity = 8192;

insert into pbank.data_zad2
values
('1', '2019-04-21', 'X'),
('1', '2019-07-01', ''),
('1', '2019-05-19', 'X'),
('1', '2019-05-01', 'X'),
('2', '2019-05-12', 'X'),
('2', '2019-05-24', ''),
('2', '2019-05-18', 'X'),
('2', '2019-05-30', 'X'),
('3', '2019-02-21', 'X'),
('3', '2019-05-15', 'X'),
('3', '2019-05-05', ''),
('3', '2019-02-21', ''),
('3', '2019-05-11', 'X');
```
<br/>
<br/>
Код SQL запроса:
<br/>

``` mysql
select 
    id as customer_id,
    min(Date) as date_min,
    max(Date) as date_max
from 
    data_zad2
where Marker = 'X'
group by
    id
order by customer_id
```
<br/>
<br/>
Результирующая таблица на скрине внизу:
<br/>

![02.jpg](pics/02.jpg)

## Задание №3: 
<br/>
Дана таблица:

***communication***

client_id | comm_date | channel
:---------|:--------- |:-------
1 | 2020-04-09 | SMS 
2 | 2020-03-17 | Call
3 | 2020-02-23 | Mail
4 | 2020-03-31 | SMS
5 | 2020-02-08 | Call
1 | 2020-04-19 | Mail
2 | 2020-04-23 | SMS
3 | 2020-04-09 | Call
4 | 2020-03-17 | Mail
5 | 2020-02-23 | SMS
1 | 2020-03-31 | Call
2 | 2020-02-08 | Mail
3 | 2020-04-19 | SMS
4 | 2020-04-23 | Call
5 | 2020-04-09 | Mail
1 | 2020-03-17 | SMS
2 | 2020-02-23 | Call
3 | 2020-03-31 | Mail
4 | 2020-02-08 | SMS
5 | 2020-04-19 | Call
2 | 2020-04-23 | Mail

Call блокирует коммуникации на Call на 60 дней, на SMS на 15 дней, на Mail на 7 дней.<br/> 
SMS блокирует коммуникации на Call и следующую SMS на 15 дней, на Mail на 7 дней. <br/>
Mail блокирует коммуникации на Call, SMS и Mail на 7 дней. <br/>
<br/>
Нужно написать код, который составляет таблицу, где напротив каждого клиента показываются все каналы, на которых можно отправить коммуникацию.<br/>
<br/>
Образец таблицы:

client_id | can_channel
:---------|:---------
1 | Call_SMS_Mail
2 | SMS_Mail
3 | Mail


## Решение Задания №3:
<br/>
Создаю таблицу, и наполню ее данными:

``` mysql
CREATE TABLE pbank.communication
(
client_id Int64,
comm_date Date,
channel String
) ENGINE = MergeTree ORDER BY (client_id) SETTINGS index_granularity = 8192;

insert into pbank.communication
values
('1', '2020-04-09', 'SMS'),
('2', '2020-03-17', 'Call'),
('3', '2020-02-23', 'Mail'),
('4', '2020-03-31', 'SMS'),
('5', '2020-02-08', 'Call'),
('1', '2020-04-19', 'Mail'),
('2', '2020-04-23', 'SMS'),
('3', '2020-04-09', 'Call'),
('4', '2020-03-17', 'Mail'),
('5', '2020-02-23', 'SMS'),
('1', '2020-03-31', 'Call'),
('2', '2020-02-08', 'Mail'),
('3', '2020-04-19', 'SMS'),
('4', '2020-04-23', 'Call'),
('5', '2020-04-09', 'Mail'),
('1', '2020-03-17', 'SMS'),
('2', '2020-02-23', 'Call'),
('3', '2020-03-31', 'Mail'),
('4', '2020-02-08', 'SMS'),
('5', '2020-04-19', 'Call'),
('2', '2020-04-23', 'Mail');
```
<br/>
<br/>
В ClickHouse нет некоторых полезных функций. Например, pivot. Но это не мешает решать поставленные задачи. 
<br/>
Код SQL запроса:
<br/>

``` mysql
with
    toDate('2020-04-25') as today_date

select
   client_id,
   concat(can_call, can_sms, can_mail) as can_channel
from
    (
    select
        client_id,
        CASE
        WHEN last_call_distance > 60 and last_sms_distance > 15 and last_mail_distance > 7 THEN 'Call'
        ELSE ''
        END as can_call,
        CASE
        WHEN last_call_distance > 15 and last_sms_distance > 15 and last_mail_distance > 7 THEN ' SMS'
        ELSE ''
        END as can_sms,
        CASE
        WHEN last_call_distance > 7 and last_sms_distance > 7 and last_mail_distance > 7 THEN ' Mail'
        ELSE ''
        END as can_mail
    from
        (
        select
            client_id,
            max(last_call_) as last_call,
            max(last_sms_) as last_sms,
            max(last_mail_) as last_mail,
            today_date - max(last_call_) as last_call_distance,
            today_date - max(last_sms_) as last_sms_distance,
            today_date - max(last_mail_) as last_mail_distance
        from
            (
            select 
                client_id,
                channel,
                CASE
                WHEN channel = 'Call' THEN max(comm_date)
                ELSE Null
                END as last_call_,
                CASE
                WHEN channel = 'SMS' THEN max(comm_date)
                ELSE Null
                END as last_sms_,
                CASE
                WHEN channel = 'Mail' THEN max(comm_date)
                ELSE Null
                END as last_mail_
            from 
                pbank.communication
            group by
                client_id, channel
            )
        group by client_id
        )
    )
order by client_id
```
<br/>
<br/>
Результирующая таблица на присланных данных и при условии что текущая дата 25.04.2020:
<br/>

![030.jpg](pics/030.jpg)

<br/>
<br/>
Результирующая таблица на присланных данных и при условии что текущая дата 25.05.2020:
<br/>

![031.jpg](pics/031.jpg)

<br/>
<br/>
Результирующая таблица на присланных данных и при условии что текущая дата 25.06.2020:
<br/>

![032.jpg](pics/032.jpg)

## Задание №4: 
<br/>
Есть двоичное число: 1000000000.<br/>
Необходимо написать запрос, который собирает все предыдущие итерации до двоичного числа 1111111111.


## Решение Задания №4:
<br/>
На MS SQL запрос будет выглядеть примерно так:

``` mysql
CREATE TABLE bitwise (   
  a_int_value INT NOT NULL,  
  b_int_value INT NOT NULL);  
GO  
INSERT bitwise VALUES (1023, 512);  
GO  

SELECT a_int_value & b_int_value  
FROM bitwise;  
GO  
```

